In [ ]:
from langchain_community.document_loaders import (
    TextLoader, UnstructuredPDFLoader, UnstructuredHTMLLoader,
    Docx2txtLoader
)

def convert_to_text(file_path):
    """The function converts a document file to text"""
    file_type = file_path.split('.')[-1]

    if "pdf" == file_type:
        loader = UnstructuredPDFLoader(file_path)
        document = loader.load()
    elif "txt" == file_type:
        loader = TextLoader(file_path)
        document = loader.load()
    elif "html" == file_type:
        loader = UnstructuredHTMLLoader(file_path)
        document = loader.load()
    elif "doc" == file_type or "docx" == file_type:
        loader = Docx2txtLoader(file_path)
        document = loader.load()
    else:
        return []
    return document


In [ ]:
docs = convert_to_text("2023_КП-МСШI_Малайдах 20230629 2.docx")
print(docs)

In [ ]:
from langchain_text_splitters import CharacterTextSplitter

splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=1000,
    chunk_overlap=200
)
split_docs = splitter.split_documents(docs)
print(split_docs)

In [ ]:
from langchain_openai.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large"  # або інша доступна модель
)


In [ ]:
from langchain_community.vectorstores import FAISS

# Індексування усіх шматків у FAISS
vectorstore = FAISS.from_texts(
    [doc.page_content for doc in split_docs],
    embeddings
)
# Перетворимо на retriever для подальшого пошуку
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})


In [ ]:
from langchain.chains.summarize import load_summarize_chain
from langchain.llms import OpenAI

# Ініціалізація LLM для сумаризації
llm = OpenAI(temperature=0)

# Створюємо chain типу map_reduce
chain = load_summarize_chain(
    llm,
    chain_type="map_reduce",
    question_prompt=None,      # не потрібен запит
    map_prompt=None,           # використовуються дефолтні шаблони
    reduce_prompt=None,        # дефолтний зведений шаблон
    token_max=1500             # макс. токенів у фінальному зведенні
)


In [ ]:
# Отримуємо релевантні документи (тут просто всі кращі k=5)
docs_to_summarize = retriever.get_relevant_documents("")

# Запускаємо ланцюг сумаризації
final_summary = chain.run(docs_to_summarize)

print("🏷 Підсумок документа:")
print(final_summary)
